In [1]:
from typing import Callable
import doctest

import numpy as np
import matplotlib.pyplot as plt

### Test functions

In [2]:
QUINTIC_SOLUTION = 1.1673039782614185

def quintic(x: float) -> float:
    return x ** 5 - x - 1


def quintic_derivative(x: float) -> float:
    return 5 * x ** 4 - 1


def quartic(x: float) -> float:
    return 16 * x ** 4 - 8 * x + 3


def quartic_derivative(x: float) -> float:
    return 64 * x ** 3 - 8


def cubic(x: float) -> float:
    return (x - 4) * (x - 1) * (x + 3)


def cubic_derivative(x: float) -> float:
    return 3 * x ** 2 - 4 * x - 11

### Equation solvers

Implement the functions below. The bisection method has been completed for you.

We want to investigate the convergence properties of each method, so in each function body we keep a list of every point visited, *e.g.*, for the bisection method, we add the new point `c` to the list at every step. Each function should return this sequence.

In [5]:
def relative_error(a, b):
    return abs((b - a) / (b + a))


def bisection(f: Callable[[float], float], a: float, b: float, tol: float) -> [float]:
    """
    Use the bisection method to find a zero of a function f on the bracket [a, b] to relative precision tol.
    
    >>> bisection(quintic, -1.0, 2.0, 1e-10)  #doctest: +ELLIPSIS
    [..., 1.16730397...]
    >>> bisection(cubic, 3.0, 5.0, 1e-10)
    [3.0, 5.0, 4.0]
    >>> bisection(quintic, 2.0, 3.0, 1e-10)
    Traceback (most recent call last):
    ValueError: [a, b] does not bracket a zero; f(a)*f(b) = 6931.0
    
    """
    fa, fb = f(a), f(b)

    if fa * fb >= 0:
        raise ValueError(f"[a, b] does not bracket a zero; f(a)*f(b) = {f(a)*f(b)}")
    
    sequence = [a, b]
    
    while relative_error(a, b) > tol:
        c = (a + b) / 2
        fc = f(c)
        if fc == 0:
            # if an exact solution is found, we should exit immediately
            sequence.append(c)
            return sequence
        
        a, b, fa, fb = (a, c, fa, fc) if fa * fc < 0.0 else (c, b, fc, fb)
        sequence.append(c)
        
    return sequence


def secant(f: Callable[[float], float], x0: float, x1: float, tol: float) -> [float]:
    """
    Use the secant method to find a zero of a function f close [x0, x1] to relative precision tol.

    >>> secant(quintic, 2.0, 3.0, 1e-10)  #doctest: +ELLIPSIS
    [..., 1.16730397...]
    >>> secant(quintic, -2.5, -2.4, 1e-8)  #doctest: +ELLIPSIS
    [..., 0.66626933...]
    """
    
    sequence = [x0, x1]
    
    while relative_error(x0, x1) > tol:
        x0, x1 = x1, x1 - f(x1) * (x1 - x0) / (f(x1) - f(x0))
        sequence.append(x1)
        
    return sequence
    
    
def newton_raphson(f: Callable[[float], float], df: Callable[[float], float], x0: float, tol: float) -> float:
    """
    Use the Newton--Raphson method to find a zero of a function f close x0 to relative precision tol.

    >>> newton_raphson(quintic, quintic_derivative, 3.0, 1e-10)  #doctest: +ELLIPSIS
    [..., 1.16730397...]
    >>> newton_raphson(quintic, quintic_derivative, 0.0, 1e-10)  #doctest: +ELLIPSIS
    Traceback (most recent call last):
    ValueError: Newton--Raphson got stuck in a cycle; cycle = [-1.00025756..., -0.75032182..., 0.08335709...]
    
    """    
    x1 = x0 - f(x0) / df(x0)
    
    sequence = [x0, x1]
    
    while relative_error(x0, x1) > tol:
        x0 = x1
        x1 = x0 - f(x0) / df(x0)
        if x1 in sequence and sequence[-1] != x1:
            cycle = sequence[sequence.index(x1):]
            raise ValueError(f"Newton--Raphson got stuck in a cycle; cycle = {cycle}")
            
        sequence.append(x1)
    
    return sequence
    

def inverse_quadratic_interpolation(f: Callable[[float], float], a: float, b: float, c: float, tol: float) -> [float]:
    """
    Use the IQR method to find a zero of a function f close to the points [a, b, c] to relative precision tol.

    >>> inverse_quadratic_interpolation(quintic, -1.0, 2.0, 3.0, 1e-10)  #doctest: +ELLIPSIS
    [..., 1.16730397...]
    >>> inverse_quadratic_interpolation(cubic, -3.0, 1.0, 4.0, 1e-10)
    Traceback (most recent call last):
    ValueError: two or more points have the same function value; f(a), f(b), f(c) = (0.0, -0.0, 0.0)
    
    """
    fa, fb, fc = f(a), f(b), f(c)
    
    sequence = [a, b, c]
    
    if fa == fb or fb == fc or fc == fa:
        raise ValueError(f"two or more points have the same function value; f(a), f(b), f(c) = {fa, fb, fc}")

    while relative_error(b, c) > tol:
        d = (
            a * fb * fc / (fa - fb) / (fa - fc) +
            b * fc * fa / (fb - fc) / (fb - fa) +
            c * fa * fb / (fc - fa) / (fc - fb)
        )
        a, b, c, fa, fb, fc = b, c, d, fb, fc, f(d)
        sequence.append(d)
        
    return sequence

doctest.testmod(verbose=True)

Trying:
    bisection(quintic, -1.0, 2.0, 1e-10)  #doctest: +ELLIPSIS
Expecting:
    [..., 1.16730397...]
ok
Trying:
    bisection(cubic, 3.0, 5.0, 1e-10)
Expecting:
    [3.0, 5.0, 4.0]
ok
Trying:
    bisection(quintic, 2.0, 3.0, 1e-10)
Expecting:
    Traceback (most recent call last):
    ValueError: [a, b] does not bracket a zero; f(a)*f(b) = 6931.0
ok
Trying:
    inverse_quadratic_interpolation(quintic, -1.0, 2.0, 3.0, 1e-10)  #doctest: +ELLIPSIS
Expecting:
    [..., 1.16730397...]
ok
Trying:
    inverse_quadratic_interpolation(cubic, -3.0, 1.0, 4.0, 1e-10)
Expecting:
    Traceback (most recent call last):
    ValueError: two or more points have the same function value; f(a), f(b), f(c) = (0.0, -0.0, 0.0)
ok
Trying:
    newton_raphson(quintic, quintic_derivative, 3.0, 1e-10)  #doctest: +ELLIPSIS
Expecting:
    [..., 1.16730397...]
ok
Trying:
    newton_raphson(quintic, quintic_derivative, 0.0, 1e-10)  #doctest: +ELLIPSIS
Expecting:
    Traceback (most recent call last):
    ValueErr

TestResults(failed=0, attempted=9)